In [1]:
// pull 방식의 for-each
val iter1 = object : Iterable<Int> {
    override fun iterator(): Iterator<Int> {
        return object : Iterator<Int> {
            var i = 1

            override fun hasNext(): Boolean { // data pull
                return i <= 10
            }

            override fun next(): Int {
                return i++
            }
        }
    }
}

val iter2: () -> Iterator<Int> = fun(): Iterator<Int> { 
    return object : Iterator<Int> {
        var i = 1
        override fun hasNext() = i <= 10
        override fun next() = i++
    }
}

val iter3: () -> Iterator<Int> = { 
    object : Iterator<Int> {
        var i = 1
        override fun hasNext() = i <= 10
        override fun next() = i++
    }
}

for (i in iter1) print(i)
for (i in iter2()) print(i)
for (i in iter3()) print(i)

123456789101234567891012345678910

In [2]:
// push 방식의 Observer 패턴
import java.util.Observable
import java.util.Observer
import java.util.concurrent.Executors

// publisher
class IntObservable : Observable(), Runnable {

    override fun run() {
        for (i in 1..10) {
            println("[${Thread.currentThread().name}] Im observable. push $i")
            setChanged()
            notifyObservers(i) // data push
        }
    }
}

// consumer
val observer = object : Observer {
    override fun update(o: Observable?, arg: Any?) {
        println("[${Thread.currentThread().name}] Im observer. received $arg")
    }
}

val io = IntObservable()
io.addObserver(observer)

io.run()

Executors.newSingleThreadExecutor().execute {  }

//io.run() // 동기
Executors.newSingleThreadExecutor().let {
    it.execute(io); it.shutdown() } // 비동기

println("[${Thread.currentThread()}] End.")

Thread.sleep(1000)

[Execution of code '// push 방식의 Observer...'] Im observable. push 1
[Execution of code '// push 방식의 Observer...'] Im observer. received 1
[Execution of code '// push 방식의 Observer...'] Im observable. push 2
[Execution of code '// push 방식의 Observer...'] Im observer. received 2
[Execution of code '// push 방식의 Observer...'] Im observable. push 3
[Execution of code '// push 방식의 Observer...'] Im observer. received 3
[Execution of code '// push 방식의 Observer...'] Im observable. push 4
[Execution of code '// push 방식의 Observer...'] Im observer. received 4
[Execution of code '// push 방식의 Observer...'] Im observable. push 5
[Execution of code '// push 방식의 Observer...'] Im observer. received 5
[Execution of code '// push 방식의 Observer...'] Im observable. push 6
[Execution of code '// push 방식의 Observer...'] Im observer. received 6
[Execution of code '// push 방식의 Observer...'] Im observable. push 7
[Execution of code '// push 방식의 Observer...'] Im observer. received 7
[Execution of code '// push 방식의 Ob

In [55]:
// push 방식의 pub-sub
import java.util.concurrent.Flow.*
import java.util.concurrent.TimeUnit

/**
* https://github.com/reactive-streams/reactive-streams-jvm/blob/v1.0.4/README.md#api-components
* 
* publisher 에 subscriber 를 subscribe() 하는 순간 publisher 는 ?
* 1. subscriber 의 onSubscribe() : 반드시 호출해야함.
* 2. subscriber 의 onNext() : optional
* 3. subscriber 의 onComplete(), onError() : optional 이나, 둘 중 하나만 호출.
*/
    

val executor = Executors.newFixedThreadPool(5)
val iterator = (1..15).iterator()

val publisher = Publisher<Int> { subscriber ->
    // subscribtion 이 현재는 publisher 안에 있어서 iterator 를 사용 가능하지만,
    // 이후에 scheduler 를 사용하거나, custom Subscription 이 생성자로 subscriber 를 받도록 구현하게 될 것.
    subscriber.onSubscribe(object: Subscription {
        override fun request(n: Long) {
            var loop = n

            val future = executor.submit {
                runCatching {
                    while (loop-- > 0) {
//                        throw IllegalArgumentException("에러 발생!!") // onError 에 에러를 인자로 넣어서 준다.
                        when {
                            iterator.hasNext() -> subscriber.onNext(iterator.next())
                            else -> subscriber.onComplete().also { loop = 0 }
                        }
                     }
                }.getOrElse { subscriber.onError(it) }
            }
            
//            future.cancel(false) // Future 객체를 사용하면 cancel 을 할 수 있음. 
        }

        override fun cancel() {
        }
    })
}


/**
* Subscription 의 request 를 언제 호출할지는 구현에 따라 다르다.
* 1. subscriber 의 onSubscribe 에서 호출하면서 버퍼에 저장할 수 도 있고,
* 2. onSubscribe 에서는 subsriber 의 맴버변수에 subscription 을 저장하고, onNext 에서 호출할 수도 있다.
* 3. 또는 스케줄러도 쓰는 듯.(아직 모름)
*/
val subscriber = object: Subscriber<Int> {
    private lateinit var subscription: Subscription
    private var initBufferSize = 3
    private var bufferSize = initBufferSize
    
    override fun onSubscribe(subscription: Subscription) {
        println("[${Thread.currentThread()}] onSubscribe")
        this.subscription = subscription
        subscription.request(bufferSize.toLong())
    }

    override fun onNext(data: Int) { // 데이터를 받는 메서드.
        println("[${Thread.currentThread()}] onNext. $data")
        // 사용자가 구현할 버퍼의 구현에 따라 request 갯수를 조절하는 컨셉.
        if (--bufferSize <= 0) { // 보통은 버퍼를 절반으로 유지.
            bufferSize = initBufferSize
            subscription.request(bufferSize.toLong())
        }
    }
    
    override fun onComplete() = println("[${Thread.currentThread()}] onComplete")
    override fun onError(throwable: Throwable) = println("[${Thread.currentThread()}] onError. ${throwable.message}")
}

publisher.subscribe(subscriber)

executor.awaitTermination(1000, TimeUnit.MILLISECONDS)
executor.shutdown()

[Thread[Execution of code '// push 방식의 pub-sub...',5,main]] onSubscribe
[Thread[pool-24-thread-1,5,main]] onNext. 1
[Thread[pool-24-thread-1,5,main]] onNext. 2
[Thread[pool-24-thread-1,5,main]] onNext. 3
[Thread[pool-24-thread-2,5,main]] onNext. 4
[Thread[pool-24-thread-2,5,main]] onNext. 5
[Thread[pool-24-thread-2,5,main]] onNext. 6
[Thread[pool-24-thread-3,5,main]] onNext. 7
[Thread[pool-24-thread-3,5,main]] onNext. 8
[Thread[pool-24-thread-3,5,main]] onNext. 9
[Thread[pool-24-thread-4,5,main]] onNext. 10
[Thread[pool-24-thread-4,5,main]] onNext. 11
[Thread[pool-24-thread-4,5,main]] onNext. 12
[Thread[pool-24-thread-5,5,main]] onNext. 13
[Thread[pool-24-thread-5,5,main]] onNext. 14
[Thread[pool-24-thread-5,5,main]] onNext. 15
[Thread[pool-24-thread-5,5,main]] onComplete


In [56]:
// publisher 가 subscriber 로 데이터를 날리는건 한 스레드에서 날림이 보장되는 스펙
// subscription 의 request 함수 n 개는 한개의 스레드에서 sequential 하게 호출되는 스펙
// subscription.request(n) 의 n 을 3으로 하고, publisher 의 publish 를 ExecutorService 를 이용해서 했을때 
// 아래와 같이 3개씩은 같은 스레드임이 보장됨을 볼 수 있다.

/*
[Thread[Execution of code 'import java.util.con...',5,main]] onSubscribe
[Thread[pool-22-thread-1,5,main]] onNext. 1
[Thread[pool-22-thread-1,5,main]] onNext. 2
[Thread[pool-22-thread-1,5,main]] onNext. 3
[Thread[pool-22-thread-2,5,main]] onNext. 4
[Thread[pool-22-thread-2,5,main]] onNext. 5
[Thread[pool-22-thread-2,5,main]] onNext. 6
[Thread[pool-22-thread-3,5,main]] onNext. 7
[Thread[pool-22-thread-3,5,main]] onNext. 8
[Thread[pool-22-thread-3,5,main]] onNext. 9
[Thread[pool-22-thread-4,5,main]] onNext. 10
[Thread[pool-22-thread-4,5,main]] onNext. 11
[Thread[pool-22-thread-4,5,main]] onNext. 12
[Thread[pool-22-thread-5,5,main]] onNext. 13
[Thread[pool-22-thread-5,5,main]] onNext. 14
[Thread[pool-22-thread-5,5,main]] onNext. 15
[Thread[pool-22-thread-5,5,main]] onComplete
 */